# Aviation Accidents Analysis

You are part of a consulting firm that is tasked to do an analysis of commercial and passenger jet airline safety. The client (an airline/airplane insurer) is interested in knowing what types of aircraft (makes/models) exhibit low rates of total destruction and low likelihood of fatal or serious passenger injuries in the event of an accident. They are also interested in any general variables/conditions that might be at play. Your analysis will be based off of aviation accident data accumulated from the years 1948-2023. 

Our client is only interested in airplane makes/models that are professional builds and could potentially still be active. Assume a max lifetime of 40 years for a make/model retirement and make sure to filter your data accordingly (i.e. from 1983 onwards). They would also like separate recommendations for small aircraft vs. larger passenger models. **In addition, make sure that claims that you make are statistically robust and that you have enough samples when making comparisons between groups.**


In this summative assessment you will demonstrate your ability to:
- **Use Pandas to load, inspect, and clean the dataset appropriately.**
- **Transform relevant columns to create measures that address the problem at hand.**
- conduct EDA: visualization and statistical measures to systematically understand the structure of the data
- recommend a set of airplanes and makes conforming to the client's request and identify at least *two* factors contributing to airplane safety. You must provide supporting evidence (visuals, summary statistics, tables) for each claim you make.

### Make relevant library imports

In [337]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

## Data Loading and Inspection

### Load in data from the relevant directory and inspect the dataframe.
- inspect NaNs, datatypes, and summary statistics

### General Overview of the dataset
- Using .info() to look into the columns and the values for our data

In [338]:
aviation_df = pd.read_csv('data/AviationData.csv', encoding = 'latin-1', low_memory = False)
aviation_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 88889 entries, 0 to 88888
Data columns (total 31 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Event.Id                88889 non-null  object 
 1   Investigation.Type      88889 non-null  object 
 2   Accident.Number         88889 non-null  object 
 3   Event.Date              88889 non-null  object 
 4   Location                88837 non-null  object 
 5   Country                 88663 non-null  object 
 6   Latitude                34382 non-null  object 
 7   Longitude               34373 non-null  object 
 8   Airport.Code            50132 non-null  object 
 9   Airport.Name            52704 non-null  object 
 10  Injury.Severity         87889 non-null  object 
 11  Aircraft.damage         85695 non-null  object 
 12  Aircraft.Category       32287 non-null  object 
 13  Registration.Number     87507 non-null  object 
 14  Make                    88826 non-null

In [339]:
aviation_df.describe(include = 'object')

,Event.Id,Investigation.Type,Accident.Number,Event.Date,Location,Country,Latitude,Longitude,Airport.Code,Airport.Name,...,Amateur.Built,Engine.Type,FAR.Description,Schedule,Purpose.of.flight,Air.carrier,Weather.Condition,Broad.phase.of.flight,Report.Status,Publication.Date
count,88889,88889,88889,88889,88837,88663,34382,34373,50132,52704,...,88787,81793,32023,12582,82697,16648,84397,61724,82505,75118
unique,87951,2,88863,14782,27758,219,25589,27154,10374,24870,...,2,12,31,3,26,13590,4,12,17074,2924
top,20001212X19172,Accident,CEN22LA149,1984-06-30,"ANCHORAGE, AK",United States,332739N,0112457W,NONE,Private,...,No,Reciprocating,091,NSCH,Personal,Pilot,VMC,Landing,Probable Cause,25-09-2020
freq,3,85015,2,25,434,82248,19,24,1488,240,...,80312,69530,18221,4474,49448,258,77303,15428,61754,17019


## Data Cleaning

### Filtering aircrafts and events

We want to filter the dataset to include aircraft that the client is interested in an analysis of:
- inspect relevant columns
- figure out any reasonable imputations
- filter the dataset

In [340]:
#Total Counts for the amateur and professional builds of the aircraft.

aviation_df['Amateur.Built'].value_counts()

Amateur.Built
No     80312
Yes     8475
Name: count, dtype: int64

In [341]:
#Count of NaNs within the Aircraft.Category

aviation_df['Aircraft.Category'].isna().sum()

56602

In [342]:
#Count of not NaNs within the Aircraft.Category

aviation_df['Aircraft.Category'].notna().sum()

32287

In [343]:
#Counts of all the aircraft categories

aviation_df['Aircraft.Category'].value_counts()

Aircraft.Category
Airplane             27617
Helicopter            3440
Glider                 508
Balloon                231
Gyrocraft              173
Weight-Shift           161
Powered Parachute       91
Ultralight              30
Unknown                 14
WSFT                     9
Powered-Lift             5
Blimp                    4
UNK                      2
Rocket                   1
ULTR                     1
Name: count, dtype: int64

In [344]:
aviation_df[aviation_df['Event.Date'] >= '1983'].count()

Event.Id                  85289
Investigation.Type        85289
Accident.Number           85289
Event.Date                85289
Location                  85237
Country                   85073
Latitude                  34379
Longitude                 34370
Airport.Code              48435
Airport.Name              50514
Injury.Severity           84289
Aircraft.damage           82151
Aircraft.Category         28723
Registration.Number       83907
Make                      85232
Model                     85211
Amateur.Built             85188
Number.of.Engines         79206
Engine.Type               78194
FAR.Description           28459
Schedule                  12045
Purpose.of.flight         79102
Air.carrier               16413
Total.Fatal.Injuries      73913
Total.Serious.Injuries    72808
Total.Minor.Injuries      73384
Total.Uninjured           79386
Weather.Condition         80798
Broad.phase.of.flight     58126
Report.Status             78905
Publication.Date          71520
dtype: i

In [345]:
aviation_df.head()[['Make', 'Aircraft.Category']]

,Make,Aircraft.Category
0,Stinson,NaN
1,Piper,NaN
2,Cessna,NaN
3,Rockwell,NaN
4,Cessna,NaN


### Filtering aircraft categories
- Figuring out the unique values within the aircraft category type
- Filling the missing values (i.e.NaNs) with 'Airplane'

In [346]:
#Unique values within the aircraft category
aviation_df['Aircraft.Category'].unique()


array([nan, 'Airplane', 'Helicopter', 'Glider', 'Balloon', 'Gyrocraft',
       'Ultralight', 'Unknown', 'Blimp', 'Powered-Lift', 'Weight-Shift',
       'Powered Parachute', 'Rocket', 'WSFT', 'UNK', 'ULTR'], dtype=object)

In [347]:
aviation_df['Aircraft.Category'].fillna('Airplane', inplace = True)
aviation_df['Aircraft.Category'].value_counts()

C:\Users\Aashi\AppData\Local\Temp\ipykernel_26876\3354989535.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  aviation_df['Aircraft.Category'].fillna('Airplane', inplace = True)


Aircraft.Category
Airplane             84219
Helicopter            3440
Glider                 508
Balloon                231
Gyrocraft              173
Weight-Shift           161
Powered Parachute       91
Ultralight              30
Unknown                 14
WSFT                     9
Powered-Lift             5
Blimp                    4
UNK                      2
Rocket                   1
ULTR                     1
Name: count, dtype: int64

In [348]:
#Filtering out dataframe with just the airplane data as the aircraft category

airplane_df = aviation_df[aviation_df['Aircraft.Category'] == 'Airplane']
airplane_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 84219 entries, 0 to 88888
Data columns (total 31 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Event.Id                84219 non-null  object 
 1   Investigation.Type      84219 non-null  object 
 2   Accident.Number         84219 non-null  object 
 3   Event.Date              84219 non-null  object 
 4   Location                84169 non-null  object 
 5   Country                 83998 non-null  object 
 6   Latitude                30507 non-null  object 
 7   Longitude               30499 non-null  object 
 8   Airport.Code            48307 non-null  object 
 9   Airport.Name            50802 non-null  object 
 10  Injury.Severity         83289 non-null  object 
 11  Aircraft.damage         81201 non-null  object 
 12  Aircraft.Category       84219 non-null  object 
 13  Registration.Number     82888 non-null  object 
 14  Make                    84159 non-null  obj

### Filtering out the airplane makes/models that are professional builds
- First, Look into the values and thier counts within the Amateur.Built
- Filter out the data with the Amateur.Built == 'No'

In [349]:
airplane_df['Amateur.Built'].value_counts()

Amateur.Built
No     76008
Yes     8111
Name: count, dtype: int64

In [350]:
#Filtering out the airplanes with professional builds

airplane_df = airplane_df[airplane_df['Amateur.Built'] == 'No']
airplane_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 76008 entries, 0 to 88888
Data columns (total 31 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Event.Id                76008 non-null  object 
 1   Investigation.Type      76008 non-null  object 
 2   Accident.Number         76008 non-null  object 
 3   Event.Date              76008 non-null  object 
 4   Location                75962 non-null  object 
 5   Country                 75793 non-null  object 
 6   Latitude                26610 non-null  object 
 7   Longitude               26605 non-null  object 
 8   Airport.Code            43344 non-null  object 
 9   Airport.Name            45641 non-null  object 
 10  Injury.Severity         75079 non-null  object 
 11  Aircraft.damage         73037 non-null  object 
 12  Aircraft.Category       76008 non-null  object 
 13  Registration.Number     74824 non-null  object 
 14  Make                    75963 non-null  obj

### Dropping the NaNs within the subset Publication.Date as we can't inpute this data

In [351]:
#All the NaNs should be dropped from the Publication.Date resulting in the sum of 0 NaNs.

airplane_df = airplane_df.dropna(subset = ['Publication.Date'])
airplane_df['Publication.Date'].isna().sum()

0

In [352]:
airplane_df['Publication.Date'].unique()


array(['19-09-1996', '26-02-2007', '12-09-2000', ..., '22-12-2022',
       '23-12-2022', '29-12-2022'], dtype=object)

### Event.Date
- Filter your data to include the airplanes from 1983 onwards.
- Assume a max lifetime of 40 years for a make/model retirement.

In [353]:
airplane_df['Report_Date'] = pd.to_datetime(airplane_df['Publication.Date'], format = 'mixed')
airplane_df = airplane_df.sort_values(by = ['Report_Date'])
airplane_df = airplane_df[airplane_df['Report_Date'] > '1983']
airplane_df.head()

,Event.Id,Investigation.Type,Accident.Number,Event.Date,Location,Country,Latitude,Longitude,Airport.Code,Airport.Name,...,Air.carrier,Total.Fatal.Injuries,Total.Serious.Injuries,Total.Minor.Injuries,Total.Uninjured,Weather.Condition,Broad.phase.of.flight,Report.Status,Publication.Date,Report_Date
217,20020917X02131,Accident,FTW82FQG14,1982-02-01,"PAMPA, TX",United States,NaN,NaN,NaN,NaN,...,NaN,2.0,0.0,0.0,0.0,IMC,Maneuvering,Probable Cause,01-02-1983,1983-01-02
218,20020917X02315,Accident,LAX82FVD09,1982-02-01,"SONOMA, CA",United States,NaN,NaN,SVA,SONOMA VALLEY,...,NaN,0.0,0.0,0.0,1.0,VMC,Takeoff,Probable Cause,01-02-1983,1983-01-02
219,20020917X02170,Accident,LAX82DA059,1982-02-01,"LAS VEGAS, NV",United States,NaN,NaN,VGT,N. LAS VEGAS AIR TERMINAL,...,NaN,0.0,0.0,0.0,2.0,VMC,Landing,Probable Cause,01-02-1983,1983-01-02
220,20020917X02006,Accident,FTW82DA053,1982-02-01,"LAS CRUCES, NM",United States,NaN,NaN,LCR,CRAWFORD,...,NaN,0.0,0.0,0.0,3.0,VMC,Landing,Probable Cause,01-02-1983,1983-01-02
222,20020917X01922,Accident,DEN82DA028,1982-02-01,"ROCK SPRINGS, WY",United States,NaN,NaN,NaN,NaN,...,Alpine Aviation,0.0,1.0,0.0,0.0,IMC,Cruise,Probable Cause,01-02-1983,1983-01-02


### Cleaning and constructing Key Measurables

Injuries and robustness to destruction are a key interest point for the client. Clean and impute relevant columns and then create derived fields that best quantifies what the client wishes to track. **Use commenting or markdown to explain any cleaning assumptions as well as any derived columns you create.**

**Construct metric for fatal/serious injuries**

*Hint:* Estimate the total number of passengers on each flight. The likelihood of serious / fatal injury can be estimated as a fraction from this.

In [354]:
#Displays the counts for all the injury types in the column

include_injury_cols = airplane_df.columns[airplane_df.columns.str.contains('njur')][1::]
airplane_df[include_injury_cols].isna().sum()

Total.Fatal.Injuries       9511
Total.Serious.Injuries    10411
Total.Minor.Injuries       9909
Total.Uninjured            4554
dtype: int64

### Evaluating and dropping all the NaNs 
- Checking the NaNs in the columns within the injuries stats
- Dropping those columns
- Also, filling all the empty values within the columns with 0

In [355]:
airplane_df[include_injury_cols].isna().all(axis = 1).sum()

140

In [356]:
#Excluding the NaNs in all the include_injury_cols

airplane_df = airplane_df[~airplane_df[include_injury_cols].isna().all(axis = 1)]
airplane_df[include_injury_cols].isna().sum()

Total.Fatal.Injuries       9371
Total.Serious.Injuries    10271
Total.Minor.Injuries       9769
Total.Uninjured            4414
dtype: int64

In [357]:
#Filling the missing or the empty values with the 0 using the fillna()

airplane_df.loc[:, include_injury_cols] = airplane_df[include_injury_cols].fillna(0)
airplane_df[include_injury_cols].isna().sum()

Total.Fatal.Injuries      0
Total.Serious.Injuries    0
Total.Minor.Injuries      0
Total.Uninjured           0
dtype: int64

### Evaluating the likelihood of fatal/serious injuries given an accident
- Calculate the total number of passengers
- Removing the rows where the total number of passengers is 0
- Also, calculating the fatal + serious injuries as a fraction

In [358]:
#This gives you the total number of passengers in a new column

airplane_df.loc[:, 'N_passenger'] = airplane_df.loc[:, include_injury_cols].sum(axis = 1)
airplane_df['N_passenger'].head()

217    2.0
218    1.0
219    2.0
220    3.0
222    1.0
Name: N_passenger, dtype: float64

In [359]:
#This includes only the rows where the total passenger is > 0

airplane_df = airplane_df[airplane_df['N_passenger'] > 0]

In [360]:
#For the fraction, we have the (fatal + serious)injuries/ (Total passengers)

airplane_df['S_njur_frac'] = (airplane_df['Total.Fatal.Injuries'] + airplane_df['Total.Serious.Injuries']) / airplane_df['N_passenger']
airplane_df['S_njur_frac'].head()

217    1.0
218    0.0
219    0.0
220    0.0
222    1.0
Name: S_njur_frac, dtype: float64

**Aircraft.Damage**
- identify and execute any cleaning tasks
- construct a derived column tracking whether an aircraft was destroyed or not.

In [361]:
#Looking into the Aircraft.damage

airplane_df['Aircraft.damage'].unique()

array(['Destroyed', 'Substantial', 'Minor', nan, 'Unknown'], dtype=object)

In [362]:
airplane_df['Aircraft.damage'].value_counts()

Aircraft.damage
Substantial    46048
Destroyed      12494
Minor           1854
Unknown           41
Name: count, dtype: int64

### Fixing the values within the Aircraft.damage
- Replacing nthe Unknown with the nan using np.nan
- Dropping all the NaNs in total

In [363]:
#Looking into the NaNs in the Aircraft.damage

airplane_df['Aircraft.damage'].isna().sum()

2014

In [364]:
#Rrplacing the Unknown value with nan and dropping all the NaNs

airplane_df['Aircraft.damage'] = airplane_df['Aircraft.damage'].replace({'Unknown':np.nan})
airplane_df.dropna(subset = ['Aircraft.damage'], inplace = True)
airplane_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 60396 entries, 217 to 88784
Data columns (total 34 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   Event.Id                60396 non-null  object        
 1   Investigation.Type      60396 non-null  object        
 2   Accident.Number         60396 non-null  object        
 3   Event.Date              60396 non-null  object        
 4   Location                60368 non-null  object        
 5   Country                 60252 non-null  object        
 6   Latitude                25230 non-null  object        
 7   Longitude               25224 non-null  object        
 8   Airport.Code            35965 non-null  object        
 9   Airport.Name            37414 non-null  object        
 10  Injury.Severity         60396 non-null  object        
 11  Aircraft.damage         60396 non-null  object        
 12  Aircraft.Category       60396 non-null  object   

### Create a new derived columns for the destroyed airplanes
- Apply boolean mask/ filter and convert to int where, (Destroyed(Yes) == 1)

In [365]:
airplane_df['Ap_destroyed'] = (airplane_df['Aircraft.damage'] == 'Destroyed').astype('int')
airplane_df['Ap_destroyed'].head()

217    1
218    0
219    0
220    0
222    0
Name: Ap_destroyed, dtype: int32

### Investigate the *Make* column
- Identify cleaning tasks here
- List cleaning tasks clearly in markdown
- Execute the cleaning tasks
- For your analysis, keep Makes with a reasonable number (you can put the threshold at 50 though lower could work as well)

In [366]:
#Converting all the Make str values to Capitalized version

airplane_df.loc[:, 'Make'] = airplane_df['Make'].str.title().str.strip()
make_counts = airplane_df['Make'].value_counts()
make_counts

Make
Cessna                      22023
Piper                       12103
Beech                        4417
Bell                         1348
Boeing                       1157
                            ...  
Evektor-Aerotechnik A.S.        1
Opus Motorsports Llc            1
Christen Industries Inc.        1
Walker                          1
Scott Terry G                   1
Name: count, Length: 1430, dtype: int64

In [367]:
#Setting the threshold to be 50

THRESHOLD = 50 

valid_makes = make_counts[make_counts >= THRESHOLD].index
airplane_df = airplane_df[airplane_df['Make'].isin(valid_makes)]
airplane_df['Make'].value_counts()

Make
Cessna                        22023
Piper                         12103
Beech                          4417
Bell                           1348
Boeing                         1157
                              ...  
British Aerospace                56
Pilatus                          53
American Champion Aircraft       51
Aerostar                         51
Balloon Works                    50
Name: count, Length: 72, dtype: int64

### Inspect Model column
- Get rid of any NaNs.
- Inspect the column and counts for each model/make. Are model labels unique to each make?
- If not, create a derived column that is a unique identifier for a given plane type.

In [368]:
airplane_df.dropna(subset = ['Model'], inplace = True)

In [369]:
airplane_df.groupby(['Model', 'Make']).count().loc['100']

,Event.Id,Investigation.Type,Accident.Number,Event.Date,Location,Country,Latitude,Longitude,Airport.Code,Airport.Name,...,Total.Minor.Injuries,Total.Uninjured,Weather.Condition,Broad.phase.of.flight,Report.Status,Publication.Date,Report_Date,N_passenger,S_njur_frac,Ap_destroyed
Make,,,,,,,,,,,,,,,,,,,,,
Aero Commander,33,33,33,33,33,33,16,16,16,22,...,33,33,33,25,31,33,33,33,33,33
Beech,10,10,10,10,10,10,3,3,7,7,...,10,10,10,7,9,10,10,10,10,10
Rockwell,1,1,1,1,1,1,0,0,0,0,...,1,1,1,1,1,1,1,1,1,1


In [370]:
airplane_df['make_model'] = airplane_df['Make'] + '_' + airplane_df['Model'].str.upper()
airplane_df['make_model'].value_counts()

make_model
Cessna_152                   1805
Cessna_172                   1468
Cessna_172N                   976
Piper_PA-28-140               741
Cessna_172M                   675
                             ... 
Mcdonnell Douglas_DC10-10       1
Cessna_180R                     1
Beech_D-50                      1
Fairchild_FA-24                 1
Pilatus_PC6                     1
Name: count, Length: 4887, dtype: int64

### Cleaning other columns
- there are other columns containing data that might be related to the outcome of an accident. We list a few here:
- Engine.Type
- Weather.Condition
- Number.of.Engines
- Purpose.of.flight
- Broad.phase.of.flight

Inspect and identify potential cleaning tasks in each of the above columns. Execute those cleaning tasks. 

**Note**: You do not necessarily need to impute or drop NaNs here.

In [371]:
airplane_df['Engine.Type'].value_counts()
airplane_df['Engine.Type'].replace({'Unknown' : np.nan, 'UNK' : np.nan}, inplace = True)

filtered_enginetype = airplane_df['Engine.Type'].value_counts()[airplane_df['Engine.Type'].value_counts() > 1]
airplane_df  = airplane_df[airplane_df['Engine.Type'].isin(filtered_enginetype.index)]
airplane_df['Engine.Type'].value_counts()


C:\Users\Aashi\AppData\Local\Temp\ipykernel_26876\3346295451.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  airplane_df['Engine.Type'].replace({'Unknown' : np.nan, 'UNK' : np.nan}, inplace = True)


Engine.Type
Reciprocating    46290
Turbo Prop        2409
Turbo Shaft       1553
Turbo Fan         1072
Turbo Jet          283
Name: count, dtype: int64

### Fixing Weather.Condition
- Looking for the unique values within the column
- Replacing 'UNK' and 'Unk' with the NaN using np.nan

In [372]:
#This lists out all the unique values in the Weather.Condition column

airplane_df['Weather.Condition'].unique()

array(['IMC', 'VMC', 'UNK', nan, 'Unk'], dtype=object)

In [373]:
#Replacing the unknowns with NaN

airplane_df['Weather.Condition'].replace({'UNK' : np.nan, 'Unk' : np.nan}, inplace = True)
airplane_df['Weather.Condition'].value_counts()

C:\Users\Aashi\AppData\Local\Temp\ipykernel_26876\526779062.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  airplane_df['Weather.Condition'].replace({'UNK' : np.nan, 'Unk' : np.nan}, inplace = True)


Weather.Condition
VMC    46777
IMC     3982
Name: count, dtype: int64

### Looking into the Number.of.Engines columns
- Using the .value_counts() to figure out the counts for all the values
- Excluding the values where the Number.of.Engine is not provided/ is 0.

In [374]:
airplane_df['Number.of.Engines'].value_counts()

Number.of.Engines
1.0    44110
2.0     6684
4.0      221
3.0      218
0.0        2
Name: count, dtype: int64

In [375]:
#Selecting just the values with the Number.of.Engines > 0.0

airplane_df = airplane_df[airplane_df['Number.of.Engines'] > 0.0]
airplane_df['Number.of.Engines'].unique()

array([1., 2., 3., 4.])

### Exploring the Purpose.of.flight
- List out the unique values in the Purpose.of.flight
- Replacing the unknowns with NaN

In [376]:
airplane_df['Purpose.of.flight'].unique()

array(['Personal', 'Instructional', 'Unknown', 'Executive/corporate',
       'Business', 'Aerial Application', 'Ferry', 'Aerial Observation',
       'Public Aircraft', 'Other Work Use', 'Positioning', nan,
       'Flight Test', 'Air Drop', 'Air Race/show', 'Skydiving',
       'Public Aircraft - State', 'Glider Tow', 'External Load',
       'Firefighting', 'Public Aircraft - Local', 'Banner Tow',
       'Public Aircraft - Federal', 'Air Race show', 'PUBS'], dtype=object)

In [377]:
#Replacing the Unknown

airplane_df['Purpose.of.flight'].replace({'Unknown' : np.nan}, inplace = True)
airplane_df['Purpose.of.flight'].unique()

C:\Users\Aashi\AppData\Local\Temp\ipykernel_26876\3127435971.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  airplane_df['Purpose.of.flight'].replace({'Unknown' : np.nan}, inplace = True)


array(['Personal', 'Instructional', nan, 'Executive/corporate',
       'Business', 'Aerial Application', 'Ferry', 'Aerial Observation',
       'Public Aircraft', 'Other Work Use', 'Positioning', 'Flight Test',
       'Air Drop', 'Air Race/show', 'Skydiving',
       'Public Aircraft - State', 'Glider Tow', 'External Load',
       'Firefighting', 'Public Aircraft - Local', 'Banner Tow',
       'Public Aircraft - Federal', 'Air Race show', 'PUBS'], dtype=object)

### Repeating the same procedure with Broad.phase.of.flight

In [378]:
airplane_df['Broad.phase.of.flight'].unique()

array(['Maneuvering', 'Takeoff', 'Landing', 'Cruise', 'Approach', 'Taxi',
       'Climb', 'Go-around', 'Unknown', 'Descent', 'Other', 'Standing',
       nan], dtype=object)

In [379]:
airplane_df['Broad.phase.of.flight'].replace({'Unknown' : np.nan}, inplace = True)
airplane_df['Broad.phase.of.flight'].unique()

C:\Users\Aashi\AppData\Local\Temp\ipykernel_26876\2235188972.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  airplane_df['Broad.phase.of.flight'].replace({'Unknown' : np.nan}, inplace = True)


array(['Maneuvering', 'Takeoff', 'Landing', 'Cruise', 'Approach', 'Taxi',
       'Climb', 'Go-around', nan, 'Descent', 'Other', 'Standing'],
      dtype=object)

### Column Removal
- inspect the dataframe and drop any columns that have too many NaNs

In [380]:
#Inspection of the dataframe

airplane_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 51233 entries, 217 to 88767
Data columns (total 36 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   Event.Id                51233 non-null  object        
 1   Investigation.Type      51233 non-null  object        
 2   Accident.Number         51233 non-null  object        
 3   Event.Date              51233 non-null  object        
 4   Location                51222 non-null  object        
 5   Country                 51112 non-null  object        
 6   Latitude                19859 non-null  object        
 7   Longitude               19854 non-null  object        
 8   Airport.Code            31399 non-null  object        
 9   Airport.Name            32831 non-null  object        
 10  Injury.Severity         51233 non-null  object        
 11  Aircraft.damage         51233 non-null  object        
 12  Aircraft.Category       51233 non-null  object   

In [381]:
airplane_df = airplane_df.drop(columns = ['FAR.Description', 'Air.carrier', 'Schedule', 'Latitude', 'Longitude', 'Airport.Code', 'Airport.Name'])
airplane_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 51233 entries, 217 to 88767
Data columns (total 29 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   Event.Id                51233 non-null  object        
 1   Investigation.Type      51233 non-null  object        
 2   Accident.Number         51233 non-null  object        
 3   Event.Date              51233 non-null  object        
 4   Location                51222 non-null  object        
 5   Country                 51112 non-null  object        
 6   Injury.Severity         51233 non-null  object        
 7   Aircraft.damage         51233 non-null  object        
 8   Aircraft.Category       51233 non-null  object        
 9   Registration.Number     51203 non-null  object        
 10  Make                    51233 non-null  object        
 11  Model                   51233 non-null  object        
 12  Amateur.Built           51233 non-null  object   

### Save DataFrame to csv
- its generally useful to save data to file/server after its in a sufficiently cleaned or intermediate state
- the data can then be loaded directly in another notebook for further analysis
- this helps keep your notebooks and workflow readable, clean and modularized

In [382]:
airplane_df.to_csv('data/air_cleaned.csv', index = False)